In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [2]:
from TexSoup import TexSoup
import re

def find_sections(tex):
    return [match.group(1) for match in re.finditer("\\\section\{(.*)\}", tex)]


def extract_content(node, name):
    if node:
        return ((name, node.contents[0]))


def section_split(tex):
    """returns: List of tuples (section_name, section_text)"""
    doc_text = tex
    sections = []
    # parsing with TexSoup
    # TODO: get rid of this and use regex only (brittle)
    try:
        soup = TexSoup(tex, tolerance=1)
        if title:=extract_content(soup.title, "title"):
            sections.append(title)
        if abstract:=extract_content(soup.abstract, "abstract"):
            sections.append(abstract)
        sections.append(("tables", "\n".join([str(node) for node in soup.find_all("table")]))) if soup.find_all("table") else None
    except:
        pass # could not parse tex

    # extract latex sections and corresponding text
    prev_section = "pre"

    for section in find_sections(tex):
        section_text, doc_text = doc_text.split(f"\\section{{{section}}}", 1)
        sections.append((prev_section, section_text))
        prev_section = section
    sections.append((prev_section, doc_text))
    return sections
    # except:
    #     return (("full", tex))
    


<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Luzi\AppData\Local\Temp\ipykernel_21096\500043745.py:5: SyntaxWarning: invalid escape sequence '\s'
  return [match.group(1) for match in re.finditer("\\\section\{(.*)\}", tex)]


In [3]:
import ollama


def pass_to_ollama(prompt, model):
    try:
        res = ollama.generate(model=model, prompt=prompt, options={"temperature": 0})
        return res["response"]
    except Exception as ex:
        print(ex)
        return f"ollama error: {ex}"


In [4]:
import json

from src.dataset import UNANSWERABLE


# prompt template
def extract_tdms_initial(tex, few_shot=True):
    if few_shot:
        return f"""If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.
        
        Text: {tex}
        
        Return the tasks, datasets, metrics and scores as reported in the text in a JSON array. Do not include precision information in the reported score.
        Here the formating structure of the JSON. Please use exactly this formating in your answer.
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        Lets make an example for you: Template-Based Automatic Search of Compact Semantic Segmentation Architectures... One discovered architecture achieves 63.2% mean IoU on CamVid and 67.8% on CityScapes having only 270K parameters... evaluation.
        
        The expected answer of you is:
        [
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CamVid", "Metric": Mean IoU", "Score": "63.2"}}, 
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CityScapes", "Metric": Mean IoU", "Score": "67.8"}}
        ]
        
        """
    else:
        return f"""If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.
        
        Text: {tex}
        
        Return the tasks, datasets, metrics and scores as reported in the text in a JSON array. Do not include precision information in the reported score.
        Here the formating structure of the JSON. Please use exactly this formating in your answer.
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        """
    
def extract_tdms_optimized01(tex, few_shot=True):
    if few_shot:
        return f"""I am a researcher and I want you to extract benchmark leaderboard results from the provided sequence of a scholarly article about Artificial Intelligence. Please return a JSON object with the defined structure below. I expect you to find TDMS(task, dataset, metric, score) tuples. Do not include precision information in the reported score. Please have a look at the example and the expected result below as well. Thank you.

        JSON structure:
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        Example:
        Template-Based Automatic Search of Compact Semantic Segmentation Architectures... One discovered architecture achieves 63.2% mean IoU on CamVid and 67.8% on CityScapes having only 270K parameters... evaluation.
        
        Result:
        [
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CamVid", "Metric": Mean IoU", "Score": "63.2"}}, 
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CityScapes", "Metric": Mean IoU", "Score": "67.8"}}
        ]
        
        Sequence: 
        {tex}
        
        """
    else:
        return f"""I am a researcher and I want you to extract benchmark leaderboard results from the provided sequence of a scholarly article about Artificial Intelligence. Please return a JSON object with the defined structure below. I expect you to find TDMS(task, dataset, metric, score) tuples. Do not include precision information in the reported score. Thank you.

        JSON structure:
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        
        Sequence: 
        {tex}
        
        """
    
def extract_tdms_optimized02(tex, few_shot=True):
    if few_shot:
        return f"""I am a researcher and I want you to extract benchmark leaderboard results from the provided sequence of a scholarly article about Artificial Intelligence. Please return a JSON object with the defined structure below. I expect you to find TDMS(task, dataset, metric, score) tuples. Most common tasks are image classification, atari games, node classification, object detection, video retrieval, link prediction, semantic segmentation, semi-supervised video object segmentation, 3d human pose estimation and question answering. Most common datasets are imagenet, coco test-dev, human3.6m, cifar-10, coco minival, youtube-vos 2018, cifar-100, msr-vtt-1ka, fb15k-237 and msu super-resolution for video compression. Most common metrics are accuracy, score, f1, psnr, map, miou, ssim, top 1 accuracy, 1:1 accuracy and number of params. Do not include precision information in the reported score. Please have a look at the example and the expected result below as well. Thank you.

        JSON structure:
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        Example:
        Template-Based Automatic Search of Compact Semantic Segmentation Architectures... One discovered architecture achieves 63.2% mean IoU on CamVid and 67.8% on CityScapes having only 270K parameters... evaluation.
        
        Result:
        [
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CamVid", "Metric": Mean IoU", "Score": "63.2"}}, 
            {{"Task": "Compact Sementic Segmentation", "Dataset": "CityScapes", "Metric": Mean IoU", "Score": "67.8"}}
        ]
        
        Sequence: 
        {tex}
        
        """
    else:
        return f"""I am a researcher and I want you to extract benchmark leaderboard results from the provided sequence of a scholarly article about Artificial Intelligence. Please return a JSON object with the defined structure below. I expect you to find TDMS(task, dataset, metric, score) tuples. Most common tasks are image classification, atari games, node classification, object detection, video retrieval, link prediction, semantic segmentation, semi-supervised video object segmentation, 3d human pose estimation and question answering. Most common datasets are imagenet, coco test-dev, human3.6m, cifar-10, coco minival, youtube-vos 2018, cifar-100, msr-vtt-1ka, fb15k-237 and msu super-resolution for video compression. Most common metrics are accuracy, score, f1, psnr, map, miou, ssim, top 1 accuracy, 1:1 accuracy and number of params. Do not include precision information in the reported score. Thank you.

        JSON structure:
        [
            {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
            {{"Task": "example Task 1", "Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
        ]
        
        Sequence: 
        {tex}
        
        """


zero_shot_template_initial = lambda tex: extract_tdms_initial(tex, False)
few_shot_template_initial = lambda tex: extract_tdms_initial(tex, True)
zero_shot_template_optimized01 = lambda tex: extract_tdms_optimized01(tex, False)
few_shot_template_optimized01 = lambda tex: extract_tdms_optimized01(tex, True)
zero_shot_template_optimized02 = lambda tex: extract_tdms_optimized02(tex, False)
few_shot_template_optimized02 = lambda tex: extract_tdms_optimized02(tex, True)

def convert_tdms_to_tuple(model_output_parsed):
    tuples = []
    for item in model_output_parsed:
        try:
            t = ((item["Task"], item["Dataset"],item["Metric"],item["Score"]))
            tuples.append(t)
        except:
            # parse error, ignore instance
            pass
    return tuples

def format_tdms(tuples):
    """make unique, format as string"""
    unique = set(tuples)
    dicts = [{"LEADERBOARD": {
        "Task": t,
        "Dataset":d,
        "Metric":m,
        "Score":s
    }} for t,d,m,s in unique]
    return str(dicts)


def section_wise_extraction(tex, prompt_template, model):
    """takes a tex file, returns an annotation"""
    found_tdms = []
    sections = section_split(str(tex))
    for section_name, section_text in sections:
        prompt = prompt_template(section_text)
        response = pass_to_ollama(prompt, model)

        try:
            response = json.loads(response)
            parsed = convert_tdms_to_tuple(response)
            found_tdms= [*found_tdms, *parsed]
        except:
            pass # no tuples found in section

    # print(found_tdms)
    if not found_tdms:
        annotation = UNANSWERABLE # found_tdms are empty -> unanswerable
    else:
        # dedupe and format
        annotation = format_tdms(found_tdms)
    return annotation

In [5]:
from src.dataset import PATH
from src.experiment_runner import run
from src.helper import get_file_content_as_string
from src.helper import model_name_existing

#Create Model
'''
model_list = ollama.list()
model_name="llama3:8b"
if model_name_existing(model_list, model_name):
    model = model_name
else:
    ollama.create(model="llama3:8b", modelfile=get_file_content_as_string("src/modelfile_llama3_8b")) '''

model="llama3:70b"

# Run on Test
df_test = run(lambda tex: section_wise_extraction(tex, zero_shot_template_initial, model), PATH.VAL, "llama3_70b_zeroshot_initial")
df_test = run(lambda tex: section_wise_extraction(tex, few_shot_template_initial, model), PATH.VAL, "llama3_70b_fewshot_initial")
df_test = run(lambda tex: section_wise_extraction(tex, zero_shot_template_optimized01, model), PATH.VAL, "llama3_70b_zeroshot_optimized01")
df_test = run(lambda tex: section_wise_extraction(tex, few_shot_template_optimized01, model), PATH.VAL, "llama3_70b_fewshot_optimized01")
df_test = run(lambda tex: section_wise_extraction(tex, zero_shot_template_optimized02, model), PATH.VAL, "llama3_70b_zeroshot_optimized02")
df_test = run(lambda tex: section_wise_extraction(tex, few_shot_template_optimized02, model), PATH.VAL, "llama3_70b_fewshot_optimized02")


FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: './data/test2-zero-shot-papers'